# Your First RAG Application

In this notebook, we'll walk you through each of the components that are involved in a simple RAG application.

We won't be leveraging any fancy tools, just the OpenAI Python SDK, Numpy, and some classic Python.

> NOTE: This was done with Python 3.11.4.

> NOTE: There might be [compatibility issues](https://github.com/wandb/wandb/issues/7683) if you're on NVIDIA driver >552.44 As an interim solution - you can rollback your drivers to the 552.44.

## Table of Contents:

- Task 1: Imports and Utilities
- Task 2: Documents
- Task 3: Embeddings and Vectors
- Task 4: Prompts
- Task 5: Retrieval Augmented Generation
  - 🚧 Activity #1: Augment RAG

Let's look at a rather complicated looking visual representation of a basic RAG application.

<img src="https://i.imgur.com/vD8b016.png" />

## Task 1: Imports and Utility

We're just doing some imports and enabling `async` to work within the Jupyter environment here, nothing too crazy!

In [44]:
from aimakerspace.text_utils import TextFileLoader, CharacterTextSplitter
from aimakerspace.vectordatabase import VectorDatabase
import asyncio

In [46]:
import nest_asyncio
nest_asyncio.apply()

## Task 2: Documents

We'll be concerning ourselves with this part of the flow in the following section:

<img src="https://i.imgur.com/jTm9gjk.png" />

### Loading Source Documents

So, first things first, we need some documents to work with.

While we could work directly with the `.txt` files (or whatever file-types you wanted to extend this to) we can instead do some batch processing of those documents at the beginning in order to store them in a more machine compatible format.

In this case, we're going to parse our text file into a single document in memory.

Let's look at the relevant bits of the `TextFileLoader` class:

```python
def load_file(self):
        with open(self.path, "r", encoding=self.encoding) as f:
            self.documents.append(f.read())
```

We're simply loading the document using the built in `open` method, and storing that output in our `self.documents` list.

> NOTE: We're using blogs from PMarca (Marc Andreessen) as our sample data. This data is largely irrelevant as we want to focus on the mechanisms of RAG, which includes out data's shape and quality - but not specifically what the contents of the data are. 


In [47]:
text_loader = TextFileLoader("data/PMarcaBlogs.txt")
documents = text_loader.load_documents()
len(documents)

1

In [48]:
print(documents[0][:100])


The Pmarca Blog Archives
(select posts from 2007-2009)
Marc Andreessen
copyright: Andreessen Horow


### Splitting Text Into Chunks

As we can see, there is one massive document.

We'll want to chunk the document into smaller parts so it's easier to pass the most relevant snippets to the LLM.

There is no fixed way to split/chunk documents - and you'll need to rely on some intuition as well as knowing your data *very* well in order to build the most robust system.

For this toy example, we'll just split blindly on length.

>There's an opportunity to clear up some terminology here, for this course we will be stick to the following:
>
>- "source documents" : The `.txt`, `.pdf`, `.html`, ..., files that make up the files and information we start with in its raw format
>- "document(s)" : single (or more) text object(s)
>- "corpus" : the combination of all of our documents

As you can imagine (though it's not specifically true in this toy example) the idea of splitting documents is to break them into managable sized chunks that retain the most relevant local context.

In [49]:
text_splitter = CharacterTextSplitter()
split_documents = text_splitter.split_texts(documents)
len(split_documents)

373

Let's take a look at some of the documents we've managed to split.

In [50]:
split_documents[0:1]

['\ufeff\nThe Pmarca Blog Archives\n(select posts from 2007-2009)\nMarc Andreessen\ncopyright: Andreessen Horowitz\ncover design: Jessica Hagy\nproduced using: Pressbooks\nContents\nTHE PMARCA GUIDE TO STARTUPS\nPart 1: Why not to do a startup 2\nPart 2: When the VCs say "no" 10\nPart 3: "But I don\'t know any VCs!" 18\nPart 4: The only thing that matters 25\nPart 5: The Moby Dick theory of big companies 33\nPart 6: How much funding is too little? Too much? 41\nPart 7: Why a startup\'s initial business plan doesn\'t\nmatter that much\n49\nTHE PMARCA GUIDE TO HIRING\nPart 8: Hiring, managing, promoting, and Dring\nexecutives\n54\nPart 9: How to hire a professional CEO 68\nHow to hire the best people you\'ve ever worked\nwith\n69\nTHE PMARCA GUIDE TO BIG COMPANIES\nPart 1: Turnaround! 82\nPart 2: Retaining great people 86\nTHE PMARCA GUIDE TO CAREER, PRODUCTIVITY,\nAND SOME OTHER THINGS\nIntroduction 97\nPart 1: Opportunity 99\nPart 2: Skills and education 107\nPart 3: Where to go and wh

## Task 3: Embeddings and Vectors

Next, we have to convert our corpus into a "machine readable" format as we explored in the Embedding Primer notebook.

Today, we're going to talk about the actual process of creating, and then storing, these embeddings, and how we can leverage that to intelligently add context to our queries.

### OpenAI API Key

In order to access OpenAI's APIs, we'll need to provide our OpenAI API Key!

You can work through the folder "OpenAI API Key Setup" for more information on this process if you don't already have an API Key!

In [51]:
import os
import openai
from getpass import getpass

openai.api_key = getpass("OpenAI API Key: ")
os.environ["OPENAI_API_KEY"] = openai.api_key

### Vector Database

Let's set up our vector database to hold all our documents and their embeddings!

While this is all baked into 1 call - we can look at some of the code that powers this process to get a better understanding:

Let's look at our `VectorDatabase().__init__()`:

```python
def __init__(self, embedding_model: EmbeddingModel = None):
        self.vectors = defaultdict(np.array)
        self.embedding_model = embedding_model or EmbeddingModel()
```

As you can see - our vectors are merely stored as a dictionary of `np.array` objects.

Secondly, our `VectorDatabase()` has a default `EmbeddingModel()` which is a wrapper for OpenAI's `text-embedding-3-small` model.

> **Quick Info About `text-embedding-3-small`**:
> - It has a context window of **8191** tokens
> - It returns vectors with dimension **1536**

#### ❓Question #1:

The default embedding dimension of `text-embedding-3-small` is 1536, as noted above. 

1. Is there any way to modify this dimension?
 
 ✅ Answer:
 "The text-embedding-3-small model by default produces a 1536-dimensional embedding vector for any input text​. Originally, OpenAI's embedding models had fixed vector sizes that could not be altered by users via the API. For example, the previous generation model text-embedding-ada-002 always returned a 1536-dimension vector with no option to change this. However, with the introduction of the third-generation embedding models in early 2024, OpenAI added support for user-specified embedding dimensions.
 The new embedding models were trained in a way that allows embeddings to be truncated without losing their "concept-representing properties." The API accepts a dimensions parameter to achieve this trade-off between vector size and accuracy​. For instance, OpenAI notes that even a text-embedding-3-large vector can be shortened from 3072 down to 256 dimensions and still outperform the older text-embedding-ada-002 model's 1536-dimensional embeddings​.
 
 In summary, users cannot increase the embedding size beyond the model's default, but they can reduce it. This change over time (from fixed to flexible dimensions) was introduced with the third-generation embedding models, as documented in OpenAI's update logs."

2. What technique does OpenAI use to achieve this?

 ✅ Answer:
 The OpenAI embeddings documentation confirms that you can request reduced-dimensional embeddings via the dimensions parameter and implies this works because of a custom training approach. However, it does not provide an explicit explanation of the underlying technique in the guide – the exact method (e.g. a particular dimensionality reduction technique or architecture change) is not documented in the public guide. All that is stated is that the models were trained in a way that allows truncating the embedding vector without significant loss of information​. Thus, the precise mechanism enabling this feature (often referred to externally as a “Matryoshka” or multi-scale embedding technique) is not described in OpenAI’s guide itself.

> NOTE: Check out this [API documentation](https://platform.openai.com/docs/api-reference/embeddings/create) for the answer to question #1, and [this documentation](https://platform.openai.com/docs/guides/embeddings/use-cases) for an answer to question #2!

We can call the `async_get_embeddings` method of our `EmbeddingModel()` on a list of `str` and receive a list of `float` back! <<- List of List of Float - One list of float is a vector representation of a chunk of text >>

```python
async def async_get_embeddings(self, list_of_text: List[str]) -> List[List[float]]:
        return await aget_embeddings(
            list_of_text=list_of_text, engine=self.embeddings_model_name
        )
```

We cast those to `np.array` when we build our `VectorDatabase()`:

```python
async def abuild_from_list(self, list_of_text: List[str]) -> "VectorDatabase":
        embeddings = await self.embedding_model.async_get_embeddings(list_of_text)
        for text, embedding in zip(list_of_text, embeddings):
            self.insert(text, np.array(embedding))
        return self
```

And that's all we need to do!

In [52]:
vector_db = VectorDatabase()
vector_db = asyncio.run(vector_db.abuild_from_list(split_documents))

#### ❓Question #2:

What are the benefits of using an `async` approach to collecting our embeddings?

✅ Answer: We need not wait for the task to complete in an async call - enables parallel tasks to happen together.


> NOTE: Determining the core difference between `async` and `sync` will be useful! If you get stuck - ask ChatGPT!

So, to review what we've done so far in natural language:

1. We load source documents
2. We split those source documents into smaller chunks (documents)
3. We send each of those documents to the `text-embedding-3-small` OpenAI API endpoint
4. We store each of the text representations with the vector representations as keys/values in a dictionary

### Semantic Similarity

The next step is to be able to query our `VectorDatabase()` with a `str` and have it return to us vectors and text that is most relevant from our corpus.

We're going to use the following process to achieve this in our toy example:

1. We need to embed our query with the same `EmbeddingModel()` as we used to construct our `VectorDatabase()`
2. We loop through every vector in our `VectorDatabase()` and use a distance measure to compare how related they are
3. We return a list of the top `k` closest vectors, with their text representations

There's some very heavy optimization that can be done at each of these steps - but let's just focus on the basic pattern in this notebook.

> We are using [cosine similarity](https://www.engati.com/glossary/cosine-similarity) as a distance metric in this example - but there are many many distance metrics you could use - like [these](https://flavien-vidal.medium.com/similarity-distances-for-natural-language-processing-16f63cd5ba55)

> We are using a rather inefficient way of calculating relative distance between the query vector and all other vectors - there are more advanced approaches that are much more efficient, like [ANN](https://towardsdatascience.com/comprehensive-guide-to-approximate-nearest-neighbors-algorithms-8b94f057d6b6)

In [53]:
vector_db.search_by_text("What is the Michael Eisner Memorial Weak Executive Problem?", k=3)

[("being IBM\nin the 80’s, or Microsoa in the 90’s, or Google today.\nHow to hire the best people you've ever worked with 73\nCuriosity\nCuriosity is a proxy for, do you love what you do?\nAnyone who loves what they do is inherently intensely curious\nabout their Xeld, their profession, their craa.\nThey read about it, study it, talk to other people about it…\nimmerse themselves in it, continuously.\nAnd work like hell to stay current in it.\nNot because they have to.\nBut because they love to.\nAnyone who isn’t curious doesn’t love what they do.\nAnd you should be hiring people who love what they do.\nAs an example, programmers.\nSit a programmer candidate for an Internet company down and\nask them about the ten most interesting things happening in\nInternet soaware.\nREST vs SOAP, the new Facebook API, whether Ruby on Rails\nis scalable, what do you think of Sun’s new Java-based scripting\nlanguage, Google’s widgets API, Amazon S3, etc.\nIf the candidate loves their Xeld, they’ll hav

## Task 4: Prompts

In the following section, we'll be looking at the role of prompts - and how they help us to guide our application in the right direction.

In this notebook, we're going to rely on the idea of "zero-shot in-context learning".

This is a lot of words to say: "We will ask it to perform our desired task in the prompt, and provide no examples."

### XYZRolePrompt

Before we do that, let's stop and think a bit about how OpenAI's chat models work.

We know they have roles - as is indicated in the following API [documentation](https://platform.openai.com/docs/api-reference/chat/create#chat/create-messages)

There are three roles, and they function as follows (taken directly from [OpenAI](https://platform.openai.com/docs/guides/gpt/chat-completions-api)):

- `{"role" : "system"}` : The system message helps set the behavior of the assistant. For example, you can modify the personality of the assistant or provide specific instructions about how it should behave throughout the conversation. However note that the system message is optional and the model’s behavior without a system message is likely to be similar to using a generic message such as "You are a helpful assistant."
- `{"role" : "user"}` : The user messages provide requests or comments for the assistant to respond to.
- `{"role" : "assistant"}` : Assistant messages store previous assistant responses, but can also be written by you to give examples of desired behavior.

The main idea is this:

1. You start with a system message that outlines how the LLM should respond, what kind of behaviours you can expect from it, and more
2. Then, you can provide a few examples in the form of "assistant"/"user" pairs
3. Then, you prompt the model with the true "user" message.

In this example, we'll be forgoing the 2nd step for simplicities sake.

#### Utility Functions

You'll notice that we're using some utility functions from the `aimakerspace` module - let's take a peek at these and see what they're doing!

##### XYZRolePrompt

Here we have our `system`, `user`, and `assistant` role prompts.

Let's take a peek at what they look like:

```python
class BasePrompt:
    def __init__(self, prompt):
        """
        Initializes the BasePrompt object with a prompt template.

        :param prompt: A string that can contain placeholders within curly braces
        """
        self.prompt = prompt
        self._pattern = re.compile(r"\{([^}]+)\}")

    def format_prompt(self, **kwargs):
        """
        Formats the prompt string using the keyword arguments provided.

        :param kwargs: The values to substitute into the prompt string
        :return: The formatted prompt string
        """
        matches = self._pattern.findall(self.prompt)
        return self.prompt.format(**{match: kwargs.get(match, "") for match in matches})

    def get_input_variables(self):
        """
        Gets the list of input variable names from the prompt string.

        :return: List of input variable names
        """
        return self._pattern.findall(self.prompt)
```

Then we have our `RolePrompt` which laser focuses us on the role pattern found in most API endpoints for LLMs.

```python
class RolePrompt(BasePrompt):
    def __init__(self, prompt, role: str):
        """
        Initializes the RolePrompt object with a prompt template and a role.

        :param prompt: A string that can contain placeholders within curly braces
        :param role: The role for the message ('system', 'user', or 'assistant')
        """
        super().__init__(prompt)
        self.role = role

    def create_message(self, **kwargs):
        """
        Creates a message dictionary with a role and a formatted message.

        :param kwargs: The values to substitute into the prompt string
        :return: Dictionary containing the role and the formatted message
        """
        return {"role": self.role, "content": self.format_prompt(**kwargs)}
```

We'll look at how the `SystemRolePrompt` is constructed to get a better idea of how that extension works:

```python
class SystemRolePrompt(RolePrompt):
    def __init__(self, prompt: str):
        super().__init__(prompt, "system")
```

That pattern is repeated for our `UserRolePrompt` and our `AssistantRolePrompt` as well.

##### ChatOpenAI

Next we have our model, which is converted to a format analagous to libraries like LangChain and LlamaIndex.

Let's take a peek at how that is constructed:

```python
class ChatOpenAI:
    def __init__(self, model_name: str = "gpt-4o-mini"):
        self.model_name = model_name
        self.openai_api_key = os.getenv("OPENAI_API_KEY")
        if self.openai_api_key is None:
            raise ValueError("OPENAI_API_KEY is not set")

    def run(self, messages, text_only: bool = True):
        if not isinstance(messages, list):
            raise ValueError("messages must be a list")

        openai.api_key = self.openai_api_key
        response = openai.ChatCompletion.create(
            model=self.model_name, messages=messages
        )

        if text_only:
            return response.choices[0].message.content

        return response
```

#### ❓ Question #3:

When calling the OpenAI API - are there any ways we can achieve more reproducible outputs?

✅ Answer:

Based on the OpenAI documentation, there are several ways to achieve more reproducible outputs when calling the OpenAI API:

*Set a seed parameter:*
Using the seed parameter allows you to get reproducible outputs for the same input
However this is currently only supported in the latest models (e.g., GPT-4 Turbo Preview and GPT-3.5 Turbo)

*Control temperature and other sampling parameters:*
Set temperature = 0 for more deterministic outputs

Lower values of temperature (closer to 0) make the output more focused and deterministic

Higher values (closer to 1) make the output more random/creative

You can also adjust top_p (nucleus sampling) for additional control

*Use system messages consistently:*
Providing the same system message helps maintain consistency in responses
System messages set the behavior and context for the model

```Python
response = client.chat.completions.create(
    model="gpt-4-turbo-preview",
    seed=123,  # Set fixed seed for reproducibility
    temperature=0,  # Most deterministic setting
    messages=[
        {"role": "system", "content": "You are a helpful assistant that provides concise answers."},
        {"role": "user", "content": "What is RAG?"}
    ]
)
```
> NOTE: Check out [this section](https://platform.openai.com/docs/guides/text-generation/) of the OpenAI documentation for the answer!

### Creating and Prompting OpenAI's `gpt-4o-mini`!

Let's tie all these together and use it to prompt `gpt-4o-mini`!

In [54]:
from aimakerspace.openai_utils.prompts import (
    UserRolePrompt,
    SystemRolePrompt,
    AssistantRolePrompt,
)

from aimakerspace.openai_utils.chatmodel import ChatOpenAI

chat_openai = ChatOpenAI()
user_prompt_template = "{content}"
user_role_prompt = UserRolePrompt(user_prompt_template)
system_prompt_template = (
    "You are an expert in {expertise}, you always answer in a kind way."
)
system_role_prompt = SystemRolePrompt(system_prompt_template)

messages = [
    system_role_prompt.create_message(expertise="Python"),
    user_role_prompt.create_message(
        content="What is the best way to write a loop?"
    ),
]

response = chat_openai.run(messages)

In [55]:
print(response)

The best way to write a loop in Python depends on what you're trying to achieve. Here are a few common ways to use loops effectively:

1. **For Loop**: This is ideal for iterating over a sequence (like a list, tuple, or a string).

   ```python
   items = ['apple', 'banana', 'cherry']
   for item in items:
       print(item)
   ```

2. **While Loop**: Use this when you want to repeat an action until a condition changes.

   ```python
   count = 0
   while count < 5:
       print(count)
       count += 1
   ```

3. **List Comprehensions**: This is a concise way to create lists. It is not a traditional loop but can achieve the same results more succinctly.

   ```python
   squares = [x**2 for x in range(10)]
   print(squares)
   ```

4. **Using Enumerate**: If you need both the index and the item from a sequence, use `enumerate()`.

   ```python
   fruits = ['apple', 'banana', 'cherry']
   for index, fruit in enumerate(fruits):
       print(f"Index {index}: {fruit}")
   ```

5. **Looping

## Task 5: Retrieval Augmented Generation

Now we can create a RAG prompt - which will help our system behave in a way that makes sense!

There is much you could do here, many tweaks and improvements to be made!

In [56]:
RAG_PROMPT_TEMPLATE = """ \
Use the provided context to answer the user's query.

You may not answer the user's query unless there is specific context in the following text.

If you do not know the answer, or cannot answer, please respond with "I don't know".
"""

rag_prompt = SystemRolePrompt(RAG_PROMPT_TEMPLATE)

USER_PROMPT_TEMPLATE = """ \
Context:
{context}

User Query:
{user_query}
"""


user_prompt = UserRolePrompt(USER_PROMPT_TEMPLATE)

class RetrievalAugmentedQAPipeline:
    def __init__(self, llm: ChatOpenAI(), vector_db_retriever: VectorDatabase) -> None:
        self.llm = llm
        self.vector_db_retriever = vector_db_retriever

    def run_pipeline(self, user_query: str) -> str:
        context_list = self.vector_db_retriever.search_by_text(user_query, k=4)

        context_prompt = ""
        for context in context_list:
            context_prompt += context[0] + "\n"

        formatted_system_prompt = rag_prompt.create_message()

        formatted_user_prompt = user_prompt.create_message(user_query=user_query, context=context_prompt)

        return {"response" : self.llm.run([formatted_system_prompt, formatted_user_prompt]), "context" : context_list}

#### ❓ Question #4:

What prompting strategies could you use to make the LLM have a more thoughtful, detailed response?

What is that strategy called?

✅ Answer:

To encourage a more thoughtful and detailed response from an LLM like GPT-3.5-turbo, you can use the following prompting strategies:

*Provide Context:*

Start with a detailed context or background information to set the stage for the response.
This helps the model understand the topic better and generate more relevant details.

*Ask Open-Ended Questions:*

Use questions that require explanation or elaboration rather than simple yes/no answers.
For example, instead of asking "Is RAG useful?", ask "How does RAG improve information retrieval?"

*Use System Messages:*

Set a system message that instructs the model to provide detailed explanations.
Example: "You are an expert in data science. Provide a comprehensive explanation."

*Chain of Thought Prompting:*

Encourage the model to think step-by-step by asking it to explain its reasoning process.
Example: "Explain the steps involved in RAG and why each step is important."

*Specify the Format:*

Request a specific format, such as bullet points or numbered lists, to organize the response.
Example: "List the benefits of RAG in bullet points."

*Use Examples:*

Provide examples in the prompt to guide the model on the level of detail expected.
Example: "Explain RAG with examples of its application in real-world scenarios."

By using these strategies, you can guide the LLM to produce more comprehensive and thoughtful responses.


> NOTE: You can look through ["Accessing GPT-3.5-turbo Like a Developer"](https://colab.research.google.com/drive/1mOzbgf4a2SP5qQj33ZxTz2a01-5eXqk2?usp=sharing) for an answer to this question if you get stuck!

In [57]:
retrieval_augmented_qa_pipeline = RetrievalAugmentedQAPipeline(
    vector_db_retriever=vector_db,
    llm=chat_openai
)

In [59]:
retrieval_augmented_qa_pipeline.run_pipeline("What is the 'Michael Eisner Memorial Weak Executive Problem'?")

{'response': "I don't know.",
 'context': [("being IBM\nin the 80’s, or Microsoa in the 90’s, or Google today.\nHow to hire the best people you've ever worked with 73\nCuriosity\nCuriosity is a proxy for, do you love what you do?\nAnyone who loves what they do is inherently intensely curious\nabout their Xeld, their profession, their craa.\nThey read about it, study it, talk to other people about it…\nimmerse themselves in it, continuously.\nAnd work like hell to stay current in it.\nNot because they have to.\nBut because they love to.\nAnyone who isn’t curious doesn’t love what they do.\nAnd you should be hiring people who love what they do.\nAs an example, programmers.\nSit a programmer candidate for an Internet company down and\nask them about the ten most interesting things happening in\nInternet soaware.\nREST vs SOAP, the new Facebook API, whether Ruby on Rails\nis scalable, what do you think of Sun’s new Java-based scripting\nlanguage, Google’s widgets API, Amazon S3, etc.\nIf t

### 🏗️ Activity #1:

Enhance your RAG application in some way! 

Suggestions are: 

- Allow it to work with PDF files
- Implement a new distance metric
- Add metadata support to the vector database

While these are suggestions, you should feel free to make whatever augmentations you desire! 

> NOTE: These additions might require you to work within the `aimakerspace` library - that's expected!

In [7]:
### YOUR CODE HERE
# Import necessary modules
from aimakerspace.vectordatabase import VectorDatabase
from aimakerspace.text_utils import TextFileLoader, CharacterTextSplitter
from aimakerspace.openai_utils.embedding import EmbeddingModel
import numpy as np
import asyncio

# Example usage
vector_db = VectorDatabase()

# Add metadata when building the database
metadata = {
    "source": "PMarcaBlogs.txt",
    "author": "Marc Andreessen",
    "date": "2007-2009"
}

# Generate embeddings for documents and add metadata
async def build_with_metadata():
    embeddings = await vector_db.embedding_model.async_get_embeddings(split_documents)
    for text, embedding in zip(split_documents, embeddings):
        vector_db.insert(text, np.array(embedding), metadata=metadata)
    return vector_db

# Run the async function
vector_db = asyncio.run(build_with_metadata())

# Create and use the pipeline
pipeline = RetrievalAugmentedQAPipeline(llm=chat_openai, vector_db_retriever=vector_db)
result = pipeline.run_pipeline("What is the Michael Eisner Memorial Weak Executive Problem?")

# Access the enhanced results
print("Response:", result["response"])
print("\nMetadata for sources:")
for item in result["metadata"]:
    print(f"\nSource: {item['metadata'].get('source')}")
    print(f"Author: {item['metadata'].get('author')}")
    print(f"Date: {item['metadata'].get('date')}")
    print(f"Relevance Score: {item['similarity_score']:.4f}")

RuntimeError: asyncio.run() cannot be called from a running event loop